In [ ]:
#!pip install ipykernel

In [1]:
#!pip install skimage.io

ERROR: Could not find a version that satisfies the requirement skimage.io (from versions: none)
ERROR: No matching distribution found for skimage.io

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: C:\Users\Braulio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import os
from tensorflow import keras
import cv2
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping

In [3]:
train_set = pd.read_csv("./train_set.csv")
train_set

,id_img,paths,label
0,0,data\train\angry\0.jpg,angry
1,1,data\train\angry\1.jpg,angry
2,10,data\train\angry\10.jpg,angry
3,10002,data\train\angry\10002.jpg,angry
4,10016,data\train\angry\10016.jpg,angry
...,...,...,...
28816,9969,data\train\surprise\9969.jpg,surprise
28817,9985,data\train\surprise\9985.jpg,surprise
28818,9990,data\train\surprise\9990.jpg,surprise
28819,9992,data\train\surprise\9992.jpg,surprise


In [4]:
test_set = pd.read_csv("./test_set.csv")
test_set

,id_img
0,10052
1,10065
2,10079
3,10095
4,10121
...,...
7061,9806
7062,9830
7063,9853
7064,9878


In [5]:
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
class_names_label

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [6]:
TRAIN_PATH = os.getcwd() + '\\data\\train'
TEST_PATH = os.getcwd() + '\\data\\test'
IMAGE_SIZE = (32,32) #para ir más rápido, que en su dia lo puse a más

In [7]:
def read_data(path, im_size):
    X = []
    Y = []

    for folder in os.listdir(path):
        label = class_names_label[folder]
        folder_path = os.path.join(path, folder)

    # Iterar sobre todo lo que haya en path
        for file in os.listdir(folder_path):
            
            # Leer la imagen a color y aplicarle el resize
            image_path = os.path.join(folder_path, file)
            image = imread(image_path)
            smallimage = cv2.resize(image, im_size)
            
            # Guardo en X
            X.append(smallimage)
            Y.append(label)

    return np.array(X), np.array(Y)

In [8]:
def read_data_test(path, im_size):
    X = []
    Y = []

    for file in os.listdir(path):
            
        # Leer la imagen a color y aplicarle el resize
        image_path = os.path.join(path, file)
        image = imread(image_path)
        smallimage = cv2.resize(image, im_size)
            
        # Guardo en X
        X.append(smallimage)
        #Y.append(label)

    return np.array(X)

In [9]:
X_train, y_train = read_data(TRAIN_PATH, IMAGE_SIZE)
X_test = read_data_test(TEST_PATH, IMAGE_SIZE)

print(X_train.shape)
print(X_test.shape)

(28821, 32, 32)
(7066, 32, 32)


In [10]:
X_train, y_train = shuffle(X_train, y_train, random_state=10)

In [11]:
pd.Series(y_train).value_counts(normalize=True)

3    0.248569
4    0.172860
5    0.171333
2    0.142361
0    0.138545
6    0.111204
1    0.015128
dtype: float64

In [12]:
X_train

array([[[ 12,  18,  34, ..., 173, 165, 156],
        [ 15,  28,  42, ..., 159, 159, 161],
        [ 16,  32,  44, ..., 172, 176, 173],
        ...,
        [168, 100,  20, ..., 181, 182, 183],
        [175, 103,  18, ..., 179, 172, 178],
        [138,  41,  23, ..., 182, 179, 183]],

       [[147,  58,  71, ..., 161, 151, 146],
        [ 46,  69,  26, ..., 167, 167, 173],
        [ 41,  80, 101, ..., 128, 157, 166],
        ...,
        [ 52,  54,  50, ...,  64,  70,  31],
        [ 57,  57,  46, ...,  63,  60,  33],
        [ 45,  62,  47, ...,  59,  42,  36]],

       [[254, 254, 254, ..., 253, 255, 255],
        [255, 254, 253, ..., 253, 254, 254],
        [255, 255, 255, ..., 246, 252, 254],
        ...,
        [249, 229, 238, ..., 166, 167, 159],
        [201, 218, 217, ..., 161, 169, 150],
        [229, 229, 220, ..., 163, 166, 147]],

       ...,

       [[203, 199, 189, ...,  17,  15,  14],
        [204, 199, 191, ...,  29,  23,  19],
        [204, 203, 186, ...,  36,  31,  27

In [13]:
model = keras.Sequential([
    keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(32,32,1)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(7, activation='softmax')
])

In [18]:
layers = [
    keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(32,32,1)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    keras.layers.Conv2D(128, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(7, activation='sigmoid')
]

#model = keras.Sequential(layers)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1152)              0         
                                                                 
 dense (Dense)               (None, 128)               1

In [20]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [21]:
earlystop = EarlyStopping(patience=5)

In [22]:
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks = [earlystop], batch_size=192)

Epoch 1/20
121/121 [==============================] - 39s 282ms/step - loss: 2.0901 - accuracy: 0.2385 - val_loss: 1.7662 - val_accuracy: 0.2959
Epoch 2/20
121/121 [==============================] - 31s 255ms/step - loss: 1.7170 - accuracy: 0.3190 - val_loss: 1.6748 - val_accuracy: 0.3518
Epoch 3/20
121/121 [==============================] - 35s 289ms/step - loss: 1.6252 - accuracy: 0.3712 - val_loss: 1.6154 - val_accuracy: 0.3807
Epoch 4/20
121/121 [==============================] - 31s 255ms/step - loss: 1.5589 - accuracy: 0.3972 - val_loss: 1.6019 - val_accuracy: 0.3795
Epoch 5/20
121/121 [==============================] - 30s 247ms/step - loss: 1.4957 - accuracy: 0.4277 - val_loss: 1.5636 - val_accuracy: 0.3972
Epoch 6/20
121/121 [==============================] - 30s 245ms/step - loss: 1.4419 - accuracy: 0.4469 - val_loss: 1.4850 - val_accuracy: 0.4359
Epoch 7/20
121/121 [==============================] - 31s 253ms/step - loss: 1.3845 - accuracy: 0.4720 - val_loss: 1.4765 - val_ac

In [23]:
df_hist = pd.DataFrame(history.history)

In [24]:
df_hist

,loss,accuracy,val_loss,val_accuracy
0,2.090119,0.238506,1.766211,0.295924
1,1.716975,0.319049,1.674812,0.351778
2,1.625231,0.371183,1.615439,0.380746
3,1.558854,0.397207,1.601877,0.379532
4,1.495728,0.427698,1.563608,0.397225
5,1.441880,0.446868,1.485007,0.435906
6,1.384469,0.471981,1.476525,0.443886
7,1.345986,0.490632,1.497912,0.439722
8,1.299632,0.511537,1.473480,0.459497
9,1.238412,0.534134,1.463813,0.464180


In [25]:
y_pred = model.predict(X_test)

221/221 [==============================] - 3s 14ms/step


In [26]:
y_pred

array([[7.93260753e-01, 2.25976779e-04, 9.70891304e-03, ...,
        4.29483540e-02, 1.07332334e-01, 4.67708160e-04],
       [1.84370596e-02, 1.19993845e-02, 3.58756006e-01, ...,
        1.60872161e-01, 3.27576935e-01, 5.09764031e-02],
       [6.78143352e-02, 2.24479707e-03, 6.64131165e-01, ...,
        2.67874803e-02, 1.04315229e-01, 7.35003278e-02],
       ...,
       [5.56984544e-02, 4.65444755e-05, 2.13923946e-01, ...,
        1.32282116e-02, 5.16952574e-02, 1.51622212e-02],
       [2.75698960e-01, 1.71210221e-03, 3.82163227e-02, ...,
        3.83555777e-02, 6.01345539e-01, 4.93549975e-03],
       [2.19359770e-01, 7.44546696e-07, 2.52641566e-05, ...,
        7.30394602e-01, 4.80445335e-03, 1.10451212e-04]], dtype=float32)

In [27]:
pred_labels = np.argmax(y_pred, axis=1)
pred_labels

array([0, 2, 2, ..., 3, 5, 4], dtype=int64)

In [28]:
label_pred = [class_names[pred] for pred in pred_labels]
label_pred

['angry',
 'fear',
 'fear',
 'surprise',
 'neutral',
 'neutral',
 'neutral',
 'fear',
 'happy',
 'happy',
 'happy',
 'angry',
 'surprise',
 'surprise',
 'neutral',
 'sad',
 'happy',
 'happy',
 'fear',
 'angry',
 'fear',
 'neutral',
 'sad',
 'sad',
 'happy',
 'fear',
 'surprise',
 'sad',
 'neutral',
 'fear',
 'angry',
 'angry',
 'happy',
 'surprise',
 'angry',
 'happy',
 'neutral',
 'fear',
 'neutral',
 'surprise',
 'happy',
 'surprise',
 'sad',
 'happy',
 'angry',
 'happy',
 'sad',
 'happy',
 'sad',
 'surprise',
 'neutral',
 'happy',
 'sad',
 'neutral',
 'surprise',
 'neutral',
 'neutral',
 'neutral',
 'fear',
 'surprise',
 'happy',
 'neutral',
 'angry',
 'angry',
 'happy',
 'neutral',
 'surprise',
 'happy',
 'fear',
 'happy',
 'surprise',
 'happy',
 'sad',
 'sad',
 'happy',
 'angry',
 'sad',
 'sad',
 'happy',
 'surprise',
 'happy',
 'happy',
 'fear',
 'fear',
 'happy',
 'happy',
 'sad',
 'sad',
 'neutral',
 'angry',
 'neutral',
 'neutral',
 'angry',
 'angry',
 'angry',
 'happy',
 'neu

In [29]:
df_submission = pd.DataFrame({"id_img": test_set["id_img"], "label": label_pred})
df_submission

,id_img,label
0,10052,angry
1,10065,fear
2,10079,fear
3,10095,surprise
4,10121,neutral
...,...,...
7061,9806,happy
7062,9830,happy
7063,9853,happy
7064,9878,sad


In [130]:
df_submission.to_csv("submission_2.csv", index = False)